In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
data_root = os.path.join('.', 'notMNIST_data') # Change me to store data elsewhere
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
def classify(dataset, checkpoint_path=None):
    """
    A sample classifier to unpickle a TensorFlow model and label the dataset.
    
    Args:
        test_dataset: the #observations by 768 feature nd-array glyphs to classify.
        checkpoint_path: optional path for previous TensorFlow session.
        
    Returns:
        The #observations by #labels nd-array labelings.
    """
    
    with tf.Session() as session:
        if not checkpoint_path:
            checkpoint_path = os.getcwd()
        saver = tf.train.import_meta_graph(os.path.join(checkpoint_path, 'model.meta'))
        saver.restore(session, os.path.join(checkpoint_path, 'model'))
        graph_predict = tf.get_default_graph()
        test_predict = graph_predict.get_tensor_by_name('hidden3/Wx_plus_b/add:0') # string from model
    
        result = test_predict.eval(feed_dict={'input/input_data:0': dataset,
                                            'dropout/Placeholder:0' : 1.0})
    return result

In [6]:
save_dir = os.path.join('notmnist_current_model', 'save')
# save_dir = os.path.join('notMNIST_data', 'save_v8')

In [7]:
predict = classify(valid_dataset, checkpoint_path=save_dir)
print("Valid accuracy: %.1f%%" % accuracy(predict, valid_labels))

INFO:tensorflow:Restoring parameters from notmnist_current_model/save/model
Valid accuracy: 90.7%


In [8]:
predict = classify(test_dataset, checkpoint_path=save_dir)
print("Test accuracy: %.1f%%" % accuracy(predict, test_labels))

INFO:tensorflow:Restoring parameters from notmnist_current_model/save/model
Test accuracy: 96.2%
